#Set-Up

In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

#Example 1
Problem with constraints and bounds

In [ ]:
#Maximize 3*x + 2*y
#such that
#x + y <= 5
#2x + y <= 3
#x,y >= 0

In [ ]:
#initialize a concrete model
model = ConcreteModel()

#define DVs
model.x = Var(domain = NonNegativeReals)
model.y = Var(domain = Reals, bounds = (0, None)) #this does the same as above

#add objective
model.obj = Objective(expr = 3*model.x + 2*model.y, sense = maximize)

#add constraint 1
model.c1 = Constraint(expr = model.x + model.y <= 5)

#add constraint 2
model.c2 = Constraint(expr = 2*model.x + model.y <= 3)

#print model
model.pprint()

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 3*x + 2*y

2 Constraint Declarations
    c1 : Size=1, Index=None, Active=True
        Key  : Lower : Body  : Upper : Active
        None :  -Inf : x + y :   5.0 :   True
    c2 : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : 2*x + y :   3.0 :   True

5 Declarations: x y obj c1 c2


In [ ]:
#solving model
opt = SolverFactory('cbc')
results = opt.solve(model, tee=False)

In [ ]:
#print solution
print("x* = ", model.x())
print("y* = ", model.y())
print("obj* = ", value(model.obj))

x* =  0.0
y* =  3.0
obj* =  6.0


#Example 2
Dealing with a bigger one using lists

In [ ]:
# Maximize 3*x1 + 2*x2 - 7*x3 + 3*x4 + 6*x5, such that
# 4*x1 + 8*x2 - 3*x3 + 5*x4 + 9*x5 <= 17
# 3*x1 + 4*x2 - 6*x3 + 1*x4 <= 19
# x1, x2, x3, x4, x5 >= 0

In [ ]:
#utilizing lists to write the above info in a scalable way
obj_coef = [3, 2, -7, 3, 6]
c1_coef = [4, 8, -3, 5, 9]
c2_coef = [3, 4, -6, 1, 0]
c1_rhs = 17
c2_rhs = 19
n = 5

In [ ]:
#define another model2 using pyomo
model2 = ConcreteModel()

#define decision variables
model2.x = Var(range(n), domain = NonNegativeReals) #creates 5 decision variables x[0], x[1], x[2], x[3] & x[4]

#define objective
model2.obj = Objective(expr = sum(obj_coef[i]*model2.x[i] for i in range(n)), sense = maximize)

#define constraints
model2.c1 = Constraint(expr = sum(c1_coef[i]*model2.x[i] for i in range(n)) <= c1_rhs)
model2.c2 = Constraint(expr = sum(c2_coef[i]*model2.x[i] for i in range(n)) <= c2_rhs)

model2.pprint()

In [ ]:
#solve
opt = SolverFactory('cbc')
results = opt.solve(model2, tee=False)

In [ ]:
#print results
for i in range(n):
  print(f"x{i} is {value(model2.x[i])}")

print("obj* = ", model2.obj())

x0 is 4.25
x1 is 0.0
x2 is 0.0
x3 is 0.0
x4 is 0.0
obj* =  12.75
